In [26]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [27]:
dataMatch = pd.read_csv("./2020_Problem_D_DATA/matchesNEW.csv")
dataMatch.rename(columns={'Outcome':'GoalDiff'}, inplace=True)
dataMatch['OpponentID'] = dataMatch['OpponentID'].str.get(8).apply(int)
dataMatch['GoalDiff'] = dataMatch['OwnScore'] - dataMatch['OpponentScore']
dataMatch['Side'] = dataMatch['Side'].apply(lambda x:1 if x == "home" else -1)
dataMatch['CoachID'] = dataMatch['CoachID'].str.get(5).apply(int)
dataMatch

,MatchID,OpponentID,GoalDiff,OwnScore,OpponentScore,Side,CoachID
0,33,1,0,0,0,1,3
1,1,1,1,1,0,1,1
2,2,2,0,1,1,-1,1
3,3,3,-2,0,2,-1,1
4,4,4,-3,0,3,1,1
5,5,5,-4,0,4,-1,1
6,6,6,1,2,1,1,1
7,7,7,-1,0,1,1,1
8,8,8,0,1,1,-1,1
9,9,9,-3,2,5,1,1


In [28]:
dataColumns = ['Ratings', 'Total Shots', 'Possession%', 'Pass Success%', 'Passes', 'Dribbles', 'Aerials Won', 'Tackles', 'Corners', 'Dispossessed', 'Interceptions', 'Clearances', 'Blocks', 'Offsides', 'Fouls', 'Touches', 'Loss of possession', 'Saves']
# data = pd.DataFrame(index=range(0,38), columns=list(dataMatch.columns)+dataColumns,dtype=float)
for i in dataColumns:
    dataMatch[i] = 0.0
dataColumnsDict = dict(zip(dataColumns, range(len(dataColumns))))
dataColumnsVis  = [0 for i in range(len(dataColumns))]
# data

In [29]:
def extractLine(line):
    words = line.strip().split()
    if (len(words)==1 and words[0] in dataColumns):
        return words[0]
    elif (len(words)==2 and ((words[0]+' '+words[1]) in dataColumns)):
        return (words[0]+' '+words[1])
    elif (len(words)==3 and ((words[0]+' '+words[1]+' '+words[2]) in dataColumns)):
        return (words[0]+' '+words[1]+' '+words[2])
    return ''
def findkeys(file, i):
    f = open(file, encoding='UTF-16')
    line = f.readline()
    while (line):
        key = extractLine(line)
        if (key!='' and dataColumnsVis[dataColumnsDict[key]] == 0):
            dataColumnsVis[dataColumnsDict[key]] = 1
            line = f.readline()
            nums = line.strip().split('-')
            if (key not in dataMatch.columns):
                dataMatch[key] = 0.0
            if (dataMatch['Side'][i] == 1):
                dataMatch.loc[i,key] = float(nums[0])
            else:
                dataMatch.loc[i,key] = float(nums[1])
        key = ''
        line = f.readline()
    f.close()
    return
for i in range(0,38):
    dataColumnsVis  = [0 for i in range(len(dataColumns))]
    file1 = "./match centre/txt"+str(38-i)+".txt"
    file2 = "./chalkboard/txt"  +str(38-i)+".txt"
    findkeys(file1, i)
    findkeys(file2, i)
data = dataMatch

In [60]:
data = data.sort_index(by=['MatchID'], axis=0, ascending = True)
data.index = range(38)
data = data.drop(['MatchID'], axis=1)
data

,OpponentID,GoalDiff,OwnScore,OpponentScore,Side,CoachID,Ratings,Total Shots,Possession%,Pass Success%,...,Corners,Dispossessed,Interceptions,Clearances,Blocks,Offsides,Fouls,Touches,Loss of possession,Saves
0,1,1,1,0,1,1,6.99,9.0,61.6,79.0,...,6.0,12.0,19.0,43.0,10.0,2.0,13.0,702.0,30.0,1.0
1,2,0,1,1,-1,1,6.50,7.0,34.6,66.0,...,1.0,7.0,16.0,30.0,11.0,3.0,9.0,477.0,17.0,5.0
2,3,-2,0,2,-1,1,6.37,7.0,43.3,81.0,...,3.0,11.0,16.0,28.0,13.0,0.0,7.0,625.0,22.0,5.0
3,4,-3,0,3,1,1,6.23,12.0,50.7,81.0,...,7.0,10.0,15.0,22.0,9.0,4.0,14.0,664.0,25.0,3.0
4,5,-4,0,4,-1,1,6.05,7.0,50.8,81.0,...,1.0,11.0,15.0,29.0,11.0,3.0,14.0,682.0,23.0,3.0
5,6,1,2,1,1,1,6.86,12.0,50.6,78.0,...,4.0,6.0,21.0,24.0,11.0,3.0,12.0,623.0,21.0,3.0
6,7,-1,0,1,1,1,6.56,25.0,63.2,79.0,...,9.0,12.0,4.0,43.0,12.0,1.0,14.0,705.0,23.0,1.0
7,8,0,1,1,-1,1,6.83,13.0,53.1,78.0,...,1.0,12.0,8.0,31.0,15.0,1.0,10.0,665.0,29.0,2.0
8,9,-3,2,5,1,1,6.30,9.0,32.4,69.0,...,0.0,8.0,10.0,30.0,22.0,2.0,14.0,477.0,22.0,9.0
9,1,-2,0,2,-1,2,6.36,15.0,59.6,80.0,...,10.0,10.0,9.0,25.0,7.0,1.0,10.0,670.0,29.0,1.0
